<a href="https://colab.research.google.com/github/Nilufayeasmin299/Reproduce-GNN_Ownership_Verification/blob/main/GNN_Ownership_using_Citeseer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install torch_geometric
!pip install numpy scikit-learn tqdm pyyaml argparse


Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.6 MB/s eta 0:00:00


In [ ]:
import torch
import numpy as np

# Configuration
config = {
    'dataset': 'Citeseer',
    'hidden_dims': [128, 64],  # Hidden layer dimensions
    'embedding_dim': 128,
    'epochs': 100,
    'learning_rate': 0.01,
    'mask_ratio': 0.2,  # Ratio of features to mask
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
}

# Set Random Seeds for Reproducibility
torch.manual_seed(42)
np.random.seed(42)
if config['device'] == 'cuda':
    torch.cuda.manual_seed_all(42)


In [ ]:
def mask_features(data, mask_ratio):
    """Apply feature masking to enrich the model's fingerprint."""
    num_features = data.x.shape[1]
    mask = np.random.choice([0, 1], size=num_features, p=[mask_ratio, 1 - mask_ratio])
    data.x = data.x * torch.tensor(mask, dtype=torch.float32, device=config['device'])
    return data


In [ ]:
from torch_geometric.nn import GCNConv, GINConv, GATConv, SAGEConv, SGConv

class GNNModel(torch.nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim, model_type='gcn'):
        super(GNNModel, self).__init__()
        if model_type == 'gcn':
            self.conv1 = GCNConv(input_dim, hidden_dims[0])
            self.conv2 = GCNConv(hidden_dims[0], hidden_dims[1])
        elif model_type == 'gin':
            self.conv1 = GINConv(torch.nn.Linear(input_dim, hidden_dims[0]))
            self.conv2 = GINConv(torch.nn.Linear(hidden_dims[0], hidden_dims[1]))
        elif model_type == 'gat':
            self.conv1 = GATConv(input_dim, hidden_dims[0])
            self.conv2 = GATConv(hidden_dims[0], hidden_dims[1])
        elif model_type == 'sage':
            self.conv1 = SAGEConv(input_dim, hidden_dims[0])
            self.conv2 = SAGEConv(hidden_dims[0], hidden_dims[1])
        elif model_type == 'sgc':
            self.conv1 = SGConv(input_dim, hidden_dims[0])
            self.conv2 = SGConv(hidden_dims[0], hidden_dims[1])
        else:
            raise ValueError(f"Unsupported model type: {model_type}")
        self.fc = torch.nn.Linear(hidden_dims[1], output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = torch.relu(self.conv1(x, edge_index))
        x = torch.relu(self.conv2(x, edge_index))
        x = self.fc(x)
        return torch.log_softmax(x, dim=1)


In [ ]:
from sklearn.metrics import accuracy_score

def train_model(model, data, optimizer, criterion, epochs):
    """Train the GNN model."""
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(data)
        loss = criterion(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()

def test_model(model, data):
    """Evaluate the model's accuracy on the test set."""
    model.eval()
    out = model(data)
    pred = out.argmax(dim=1)
    acc = accuracy_score(data.y[data.test_mask].cpu(), pred[data.test_mask].cpu())
    return acc


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

def evaluate_classifier(target_model, shadow_model, data):
    """Train and evaluate the ownership verification classifier."""
    target_model.eval()
    shadow_model.eval()

    target_embeddings = target_model(data).detach().cpu().numpy()
    shadow_embeddings = shadow_model(data).detach().cpu().numpy()

    X = np.vstack([target_embeddings, shadow_embeddings])
    y = np.array([1] * len(target_embeddings) + [0] * len(shadow_embeddings))  # 1 = Extracted, 0 = Independent

    # Split data for training and testing
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train ownership classifier
    clf = RandomForestClassifier()
    clf.fit(X_train, y_train)

    # Evaluate classifier
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)

    # Confusion Matrix
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
    fnr = fn / (fn + tp) if (fn + tp) > 0 else 0

    return acc, fpr, fnr


In [ ]:
from torch_geometric.datasets import Planetoid

# Load Dataset
dataset = Planetoid(root='./data', name=config['dataset'])
data = dataset[0].to(config['device'])

# Apply Feature Masking
data = mask_features(data, config['mask_ratio'])

# Evaluate with Multiple GNN Architectures
architectures = ['gcn', 'gin', 'gat', 'sage', 'sgc']
results = {}

for arch in architectures:
    print(f"Evaluating {arch.upper()} Model")

    # Initialize Models
    target_model = GNNModel(dataset.num_features, config['hidden_dims'], dataset.num_classes, arch).to(config['device'])
    shadow_model = GNNModel(dataset.num_features, config['hidden_dims'], dataset.num_classes, arch).to(config['device'])

    # Train Models
    optimizer = torch.optim.Adam(target_model.parameters(), lr=config['learning_rate'])
    criterion = torch.nn.NLLLoss()

    train_model(target_model, data, optimizer, criterion, config['epochs'])
    train_model(shadow_model, data, optimizer, criterion, config['epochs'])

    # Test Target Model
    test_accuracy = test_model(target_model, data)
    print(f"Target Model Test Accuracy ({arch.upper()}): {test_accuracy:.4f}")

    # Evaluate Ownership Classifier
    acc, fpr, fnr = evaluate_classifier(target_model, shadow_model, data)
    print(f"Ownership Classifier Accuracy: {acc:.4f}, FPR: {fpr:.4f}, FNR: {fnr:.4f}")

    results[arch] = {
        'test_accuracy': test_accuracy,
        'ownership_accuracy': acc,
        'fpr': fpr,
        'fnr': fnr
    }

# Display Results
print("\nFinal Results Across Architectures:")
for arch, metrics in results.items():
    print(f"{arch.upper()} - Test Accuracy: {metrics['test_accuracy']:.4f}, Ownership Accuracy: {metrics['ownership_accuracy']:.4f}, FPR: {metrics['fpr']:.4f}, FNR: {metrics['fnr']:.4f}")


Processing...
Done!


Evaluating GCN Model
Target Model Test Accuracy (GCN): 0.6280
Ownership Classifier Accuracy: 1.0000, FPR: 0.0000, FNR: 0.0000
Evaluating GIN Model
Target Model Test Accuracy (GIN): 0.5880
Ownership Classifier Accuracy: 0.9970, FPR: 0.0030, FNR: 0.0030
Evaluating GAT Model
Target Model Test Accuracy (GAT): 0.5910
Ownership Classifier Accuracy: 1.0000, FPR: 0.0000, FNR: 0.0000
Evaluating SAGE Model
Target Model Test Accuracy (SAGE): 0.6040
Ownership Classifier Accuracy: 1.0000, FPR: 0.0000, FNR: 0.0000
Evaluating SGC Model
Target Model Test Accuracy (SGC): 0.6010
Ownership Classifier Accuracy: 1.0000, FPR: 0.0000, FNR: 0.0000

Final Results Across Architectures:
GCN - Test Accuracy: 0.6280, Ownership Accuracy: 1.0000, FPR: 0.0000, FNR: 0.0000
GIN - Test Accuracy: 0.5880, Ownership Accuracy: 0.9970, FPR: 0.0030, FNR: 0.0030
GAT - Test Accuracy: 0.5910, Ownership Accuracy: 1.0000, FPR: 0.0000, FNR: 0.0000
SAGE - Test Accuracy: 0.6040, Ownership Accuracy: 1.0000, FPR: 0.0000, FNR: 0.0000
SG